In [ ]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from deap import base, creator, tools, algorithms
import random
import multiprocessing

# Define the SewerX function representing the mechanistic model
def SewerX(t, y, kHAC, kSRBAC, kMPBAC):
    # Unpack state variables
    SF, Spro, Sac, Sh, SCH4, SSO4, SH2S, Se, XS, XI, Xaci, Xace, XMAac, XMAh, XSRBpr, XSRBac, XSRBh = y
    
    # Define constants
    khydro = 3
    kaci = 6
    kace = 6
    KF = 10
    # Use optimization parameters
    kMAac = kHAC
    KMAac = 210
    kMAh = kSRBAC
    KMAh = 0.1
    kSRBpro = kMPBAC
    KSRBpro = 110
    kSRBac = 7.1
    KSRBac = 220
    kSRBh = 26.7
    KSRBh = 0.1
    KSO4 = 1.8
    kh2s = 1
    kdecaci = 0.02
    kdecace = 0.02
    kdecMAac = 0.015
    kdecMAh = 0.01
    kdecSRBpro = 0.010
    kdecSRBac = 0.015
    kdecSRBh = 0.01
    f1 = 0.78
    f2 = 0.22
    f3 = 0.67
    f4 = 0.33
    n = 1.65
    Yaci = 0.1
    Yace = 0.06
    YMAac = 0.0317
    YMAh = 0.0403
    YSRBac = 0.0329
    YSRBpro = 0.0342
    YSRBh = 0.0366

    # Define the A matrix based on the model equations
    A = np.array([
        [1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0],
        [-1, f1*(1-Yaci), f2*(1-Yaci), 0, 0, 0, 0, 0, 0, 0, Yaci, 0, 0, 0, 0, 0, 0],
        [-1, 0, f3*(1-Yace), f4*(1-Yace), 0, 0, 0, 0, 0, 0, 0, Yace, 0, 0, 0, 0, 0],
        [0, 0, -1, 0, 1-YMAac, 0, 0, 0, 0, 0, 0, 0, YMAac, 0, 0, 0, 0],
        [0, 0, 0, -1, 1-YMAh, 0, 0, 0, 0, 0, 0, 0, 0, YMAh, 0, 0, 0],
        [0, -1, 0, 0, 0, (YSRBpro-1)/2, (1-YSRBpro)/2, 0, 0, 0, 0, 0, 0, 0, YSRBpro, 0, 0],
        [0, 0, -1, 0, 0, (YSRBac-1)/2, (1-YSRBac)/2, 0, 0, 0, 0, 0, 0, 0, 0, YSRBac, 0],
        [0, 0, 0, -1, 0, (YSRBh-1)/2, (1-YSRBh)/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, YSRBh],
        [0, 0, 0, 0, 0, 0, -1, -n, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.9, 0.1, -1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.9, 0.1, 0, -1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.9, 0.1, 0, 0, -1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.9, 0.1, 0, 0, 0, -1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.9, 0.1, 0, 0, 0, 0, -1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.9, 0.1, 0, 0, 0, 0, 0, -1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.9, 0.1, 0, 0, 0, 0, 0, 0, -1]
    ]).transpose()

    # Define the B vector based on the model equations
    B = [
        khydro * XS,
        kaci * SF / (KF + SF) * Xaci,
        kace * SF / (KF + SF) * Xace,
        kMAac * Sac / (KMAac + Sac) * XMAac,
        kMAh * Sh / (KMAh + Sh) * XMAh,
        kSRBpro * Spro * SSO4 * XSRBpr / (KSRBpro + Spro) / (KSO4 + KSO4),
        kSRBac * Sac * SSO4 * XSRBac / (KSRBac + Sac) / (KSO4 + SSO4),
        kSRBh * Sh * SSO4 * XSRBh / (KSRBh + Sh) / (KSO4 + SSO4),
        kh2s * SH2S * Se,
        kdecaci * Xaci,
        kdecace * Xace,
        kdecMAac * XMAac,
        kdecMAh * XMAh,
        kdecSRBpro * XSRBpr,
        kdecSRBac * XSRBac,
        kdecSRBh * XSRBh      
    ]

    # Calculate the derivatives based on the A matrix and B vector
    return [np.dot(A[i, :], B) for i in range(17)]

# Load the datasets
influent_data = pd.read_csv("influent.csv")
position1_data = pd.read_csv("position1.csv")
position2_data = pd.read_csv("position2.csv")
position3_data = pd.read_csv("position3.csv")
position4_data = pd.read_csv("position4.csv")

# Ensure all datasets have the same number of samples
num_samples = len(influent_data)
assert len(position1_data) == num_samples, "Position1 data rows do not match influent data."
assert len(position2_data) == num_samples, "Position2 data rows do not match influent data."
assert len(position3_data) == num_samples, "Position3 data rows do not match influent data."
assert len(position4_data) == num_samples, "Position4 data rows do not match influent data."

# Define the reaction times corresponding to each position
reaction_times = [0.5, 1.0, 1.5, 2.0]

# Define the fitness function to evaluate the model's performance
def evaluate_model(individual):
    """
    Evaluate the model by calculating the total Mean Squared Error (MSE)
    between the model predictions and the observed data across all positions and samples.
    
    Parameters:
    - individual: A list containing the parameters [kHAC, kSRBAC, kMPBAC] to be optimized.
    
    Returns:
    - A tuple containing the total MSE.
    """
    kHAC, kSRBAC, kMPBAC = individual
    total_mse = 0.0

    for index in range(num_samples):
        # Get the initial conditions for the current sample from the influent data
        y0 = influent_data.iloc[index].values

        # Define the time span and evaluation points for the ODE solver
        t_span = (0, 2.0)
        t_eval = reaction_times

        # Solve the ODEs using the SewerX function with the current parameters
        try:
            sol = solve_ivp(
                SewerX, 
                t_span, 
                y0, 
                args=(kHAC, kSRBAC, kMPBAC), 
                t_eval=t_eval, 
                method='LSODA'
            )
        except Exception as e:
            # Assign a large error if the solver fails
            return (1e6,)

        if not sol.success:
            # Assign a large error if the solver did not succeed
            return (1e6,)

        # Transpose the solution to have each row correspond to a time point
        y = sol.y.T

        # Compare the model output with observed data for each reaction time
        for i, t in enumerate(reaction_times):
            # Get the model output at the current time point
            model_output = y[i, :]

            # Retrieve the corresponding observed data based on the reaction time
            if i == 0:
                observed = position1_data.iloc[index].values
            elif i == 1:
                observed = position2_data.iloc[index].values
            elif i == 2:
                observed = position3_data.iloc[index].values
            elif i == 3:
                observed = position4_data.iloc[index].values
            else:
                continue

            # Calculate the Mean Squared Error (MSE) between model output and observed data
            mse = np.mean((model_output - observed) ** 2)
            total_mse += mse

    return (total_mse,)

# Setup the Genetic Algorithm using DEAP
# Define the fitness as minimizing the total MSE
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# Define an individual as a list with fitness attribute
creator.create("Individual", list, fitness=creator.FitnessMin)

# Initialize the toolbox
toolbox = base.Toolbox()

# Define the parameter ranges for kHAC, kSRBAC, and kMPBAC
kHAC_min, kHAC_max = 0.1, 20.0
kSRBAC_min, kSRBAC_max = 0.1, 20.0
kMPBAC_min, kMPBAC_max = 0.1, 20.0

# Register the attribute generators for each parameter
toolbox.register("attr_kHAC", random.uniform, kHAC_min, kHAC_max)
toolbox.register("attr_kSRBAC", random.uniform, kSRBAC_min, kSRBAC_max)
toolbox.register("attr_kMPBAC", random.uniform, kMPBAC_min, kMPBAC_max)

# Register the individual and population generators
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.attr_kHAC, toolbox.attr_kSRBAC, toolbox.attr_kMPBAC), n=1)
toolbox.register("population", tools.initPopulation, list, toolbox.individual)

# Register the evaluation function
toolbox.register("evaluate", evaluate_model)

# Register the genetic operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    """
    Main function to execute the Genetic Algorithm for parameter optimization.
    Implements an early stopping mechanism based on lack of improvement.
    """
    # GA parameters
    population_size = 50
    max_generations = 100  # Set a high maximum number of generations
    crossover_prob = 0.7    # Probability of mating
    mutation_prob = 0.2     # Probability of mutation
    patience = 10           # Number of generations to wait for improvement
    improvement_threshold = 1e-6  # Minimum improvement to reset patience

    # Initialize the population
    pop = toolbox.population(n=population_size)

    # Set up multiprocessing pool for parallel evaluation
    pool = multiprocessing.Pool()
    toolbox.register("map", pool.map)

    # Initialize tracking variables for early stopping
    best_fitness = None
    no_improve_count = 0
    best_individual = None

    # Evolutionary loop
    for gen in range(max_generations):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < crossover_prob:
                toolbox.mate(child1, child2)
                # Invalidate fitness values after crossover
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation on the offspring
        for mutant in offspring:
            if random.random() < mutation_prob:
                toolbox.mutate(mutant)
                # Invalidate fitness values after mutation
                del mutant.fitness.values

        # Evaluate the individuals with invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Replace the old population with the new offspring
        pop[:] = offspring

        # Gather all the fitnesses in the population
        fits = [ind.fitness.values[0] for ind in pop]
        min_fit = min(fits)
        avg_fit = np.mean(fits)
        max_fit = max(fits)

        # Print the statistics for the current generation
        print(f"Generation {gen+1}: Min Fitness = {min_fit}, Avg Fitness = {avg_fit}, Max Fitness = {max_fit}")

        # Check for improvement
        if best_fitness is None or best_fitness - min_fit > improvement_threshold:
            best_fitness = min_fit
            no_improve_count = 0
            # Update the best individual found so far
            best_individual = tools.selBest(pop, 1)[0]
        else:
            no_improve_count += 1
            print(f"No significant improvement. Patience count: {no_improve_count}/{patience}")

        # Check if early stopping condition is met
        if no_improve_count >= patience:
            print(f"No improvement in the last {patience} generations. Stopping early.")
            break

    # Close the multiprocessing pool
    pool.close()
    pool.join()

    # Select and print the best individual
    best_ind = tools.selBest(pop, 1)[0]
    print(f"Best Individual: {best_ind}")
    print(f"Best Fitness (Total MSE): {best_ind.fitness.values[0]}")

    # Save the best results to a text file
    with open("C:/Users/Van/Desktop/ga_results.txt", "w") as f:
        f.write(f"Best Individual: {best_ind}\n")
        f.write(f"Best Fitness (Total MSE): {best_ind.fitness.values[0]}\n")

if __name__ == "__main__":
    main()
